In [1]:
from arithmetic import Variable, Constant, SumFromTo
from graph import Graph
from cost_functions import *
import math

def c(value: float | int) -> Constant:
    return Constant(value)

### Arithmetic

In [1]:
x = Variable("x", ["v", "\\alpha"])
i = Variable("i")
N = Variable("N")
x_i = Variable("x", ["v", "i"])

NameError: name 'Variable' is not defined

In [3]:
((x + 1) / x * (x + 2) + SumFromTo(x_i, i, c(1), N))

Addition(order=3, left=Multiplication(order=2, left=Division(order=2, left=Addition(order=3, left=Variable(order=0, name='x', subscripts=[Variable(order=0, name='v', subscripts=[], superscripts=[])], superscripts=[Variable(order=0, name='\\alpha', subscripts=[], superscripts=[])]), right=Constant(order=0, value=1.0)), right=Variable(order=0, name='x', subscripts=[Variable(order=0, name='v', subscripts=[], superscripts=[])], superscripts=[Variable(order=0, name='\\alpha', subscripts=[], superscripts=[])])), right=Addition(order=3, left=Variable(order=0, name='x', subscripts=[Variable(order=0, name='v', subscripts=[], superscripts=[])], superscripts=[Variable(order=0, name='\\alpha', subscripts=[], superscripts=[])]), right=Constant(order=0, value=2.0))), right=SumFromTo(order=3, expression=Variable(order=0, name='x', subscripts=[Variable(order=0, name='v', subscripts=[], superscripts=[])], superscripts=[Variable(order=0, name='i', subscripts=[], superscripts=[])]), variable=Variable(ord

### Merging Cost Functions

In [4]:
with open("graph", "r") as file:
    g = Graph.read(file)
    
constraints: list[CostFunction] = [
    PathIsValid([0]),
    PathIsLoop(),
    PathContainsVerticesExactlyOnce(list(range(g.n_vertices)))
]

optimisation_goals: list[CostFunction] = [
    MinimisePathLength([0])
]

full_cost_function = merge(constraints, optimisation_goals)
#full_cost_function.get_formula(g, EncodingType.OneHot)

### UI

In [5]:
import ipywidgets as widgets

dropdown_style = {"description_width": "40%"}

with open("graph", "r") as file:
    g = Graph.read(file)

graph_text_area = widgets.Textarea(
        value=str(g.serialize()),
        disabled=False,
        layout={"height": "360px", "width": "200px"}
    )
graph_output = widgets.Output(
    
)
with graph_output:
    g.plot()
graph_update_button = widgets.Button(description="Update", tooltip="Construct a new graph from the provided adjacency matrix.", disabled=False)

def graph_update_button_click(_: widgets.Button):
    graph_output.clear_output()
    g = Graph.deserialize(graph_text_area.value)
    with open("graph", "w") as file:
        g.store(file)
    with graph_output:
        g.plot()
graph_update_button.on_click(graph_update_button_click)
graph_input = widgets.VBox([
    widgets.HBox([
        graph_text_area,
        graph_output
    ]),
    graph_update_button
])

encoding = widgets.VBox([
    widgets.Dropdown(
        options=["One-Hot", "Unary", "Binary"],
        value="One-Hot",
        description="Encoding:",
        disabled=False
    ),
    widgets.ToggleButtons(
        options=['|V|/2', '|V|', '|V| + 1', '2*|V|'],
        description='Maximum allowed Path length:',
        disabled=False,
        value="|V|",
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltips=['A path has at most |V|/2 vertices.', 'A path has at most |V|/2 vertices.', 'A path has at most 2*|V| vertices. Try to minimise as far as possible.'],
        style=dropdown_style
    ),
])

path_properties = widgets.VBox([
    widgets.HBox([
        widgets.ToggleButton(
            value=True,
            description="Valid",
            tooltip="Selected by default. When disabled, cost function will not contain term to check if path is conntected.",
            disabled=False,
            indent=False
        ),
        widgets.ToggleButton(
            value=False,
            description="Loop",
            tooltip="Indicates that the path should end at the starting position.",
            disabled=False,
            indent=False
        ),
        widgets.ToggleButton(
            value=False,
            description="Vertex-Disjoint",
            tooltip="Prevent Path intersections on Vertices (Currently not supported).",
            disabled=True,
            indent=False
        ),
        widgets.ToggleButton(
            value=False,
            description="Edge-Disjoint",
            tooltip="Prevent Path intersections on Edges (Currently not supported).",
            disabled=True,
            indent=False
        )
    ]),
    widgets.BoundedIntText(
        value=1,
        description="Number of Paths:",
        disabled=False,
        style=dropdown_style,
        min=1,
        max=9
    )
])
vertex_properties = widgets.VBox([
    widgets.Dropdown(
        options=["Any"] + list(range(1, g.n_vertices + 1)),
        value="Any",
        description="Starting Vertex:",
        disabled=False,
        style=dropdown_style
    ),
    widgets.Dropdown(
        options=["Any"] + list(range(1, g.n_vertices + 1)),
        value="Any",
        description="End Vertex:",
        disabled=False,
        style=dropdown_style
    ),
    widgets.VBox([
        widgets.Label(value="The following vertices must appear EXACTLY once per path:"),
        widgets.GridBox([
                widgets.ToggleButton(
                    value=False,
                    description=f"{i}",
                    disabled=False,
                    layout={"width": "30px"}
                )
                for i in range(1, g.n_vertices + 1)
            ],
            layout=widgets.Layout(grid_template_columns=f"repeat({int(math.ceil(g.n_vertices**0.5))}, 35px)")
        )
    ]),
    widgets.VBox([
        widgets.Label(value="The following vertices must appear AT LEAST once per path (Currently not supported):"),
        widgets.GridBox([
                widgets.ToggleButton(
                    value=False,
                    description=f"{i}",
                    disabled=False,
                    layout={"width": "30px"}
                )
                for i in range(1, g.n_vertices + 1)
            ],
            layout=widgets.Layout(grid_template_columns=f"repeat({int(math.ceil(g.n_vertices**0.5))}, 35px)")
        )
    ]),
    widgets.VBox([
        widgets.Label(value="The following vertices can appear AT MOST once per path (Currently not supported):"),
        widgets.GridBox([
                widgets.ToggleButton(
                    value=False,
                    description=f"{i}",
                    disabled=False,
                    layout={"width": "30px"}
                )
                for i in range(1, g.n_vertices + 1)
            ],
            layout=widgets.Layout(grid_template_columns=f"repeat({int(math.ceil(g.n_vertices**0.5))}, 35px)")
        )
    ])
])

optimisation_goals_input = widgets.VBox([
    widgets.ToggleButton(
        value=True,
        tooltip="When enabled, optimises the selected path(s) with respect to their lengths.",
        description="Minimise Path Length",
        disabled=False,
        indent=False
    ),
])

generate_button = widgets.Button(description="Generate", disabled=False)
def generate_click(button: widgets.Button):
    selected_encoding = encoding.children[0].value
    maximum_path_length = encoding.children[1].value
    
    is_valid = path_properties.children[0].children[0].value
    is_loop = path_properties.children[0].children[1].value
    vertex_disjoint = path_properties.children[0].children[2].value
    edge_disjoint = path_properties.children[0].children[3].value
    n_paths = path_properties.children[1].value
    
    starting_vertex = vertex_properties.children[0].value
    ending_vertex = vertex_properties.children[1].value
    exactly_once = [i + 1 for i, child in enumerate(vertex_properties.children[2].children[1].children) if child.value]
    at_least_once = [i + 1 for i, child in enumerate(vertex_properties.children[3].children[1].children) if child.value]
    at_most_once = [i + 1 for i, child in enumerate(vertex_properties.children[4].children[1].children) if child.value]
    
    minimise_path_length = optimisation_goals_input.children[0].value
    
    constraints: list[CostFunction] = []
    if is_valid:
        constraints.append(PathIsValid([0]))
    if is_loop:
        constraints.append(PathIsLoop())
    if vertex_disjoint:
        constraints.append(PathsShareNoVertices([0]))
    if edge_disjoint:
        constraints.append(PathsShareNoEdges([0]))
    
    if starting_vertex != "Any":
        constraints.append(PathStartsAt([int(starting_vertex)]))
    if ending_vertex != "Any":
        constraints.append(PathEndsAt([int(ending_vertex)]))
    if exactly_once:
        constraints.append(PathContainsVerticesExactlyOnce(exactly_once))
    if at_least_once:
        constraints.append(PathContainsVerticesAtLeastOnce(at_least_once))
    if at_most_once:
        constraints.append(PathContainsVerticesAtMostOnce(at_most_once))

    optimisation_goals: list[CostFunction] = []
    if minimise_path_length:
        optimisation_goals.append(MinimisePathLength([0]))

    full_cost_function = merge(constraints, optimisation_goals)
    display(full_cost_function.get_formula(g, EncodingType.OneHot))

generate_button.on_click(generate_click)


widgets.VBox([
    widgets.Accordion(
        children=[graph_input, encoding, path_properties, vertex_properties, optimisation_goals_input],
        titles=("Graph", "Encoding", "Path Properties", "Vertex Properties", "Optimisation Goals")
    ),
    generate_button
])

Addition(order=3, left=Multiplication(order=2, left=Constant(order=0, value=1), right=SumSet(order=3, expression=SumFromTo(order=3, expression=Multiplication(order=2, left=Variable(order=0, name='x', subscripts=[Variable(order=0, name='v', subscripts=[], superscripts=[])], superscripts=[Variable(order=0, name='i', subscripts=[], superscripts=[])]), right=Variable(order=0, name='x', subscripts=[Variable(order=0, name='w', subscripts=[], superscripts=[])], superscripts=[Variable(order=0, name='i + 1', subscripts=[], superscripts=[])])), variable=Variable(order=0, name='i', subscripts=[], superscripts=[]), from_value=Constant(order=0, value=1), to_value=Constant(order=0, value=4)), variables=[Variable(order=0, name='v', subscripts=[], superscripts=[]), Variable(order=0, name='w', subscripts=[], superscripts=[])], set_expression='\\not\\in E', set_callback=<function PathIsValid.get_formula.<locals>.<lambda> at 0x7f5447083ec0>)), right=Multiplication(order=2, left=Constant(order=0, value=1)